In [ ]:
import tensorflow as tf 
import random, numpy as np, os
def config_gpu(mp=False):
    print('Eager Model : ', tf.executing_eagerly())
    print('TensorFlow Cuda Built Test : ', tf.test.is_built_with_cuda)
    print('TensorFlow GPU Detected : ', tf.test.gpu_device_name())
    print('TensorFlow System Cuda Version : ', tf.sysconfig.get_build_info()["cuda_version"])
    print('TensorFlow System CudNN Version : ', tf.sysconfig.get_build_info()["cudnn_version"] )

    AUTO = tf.data.AUTOTUNE
    GPUS = tf.config.list_physical_devices('GPU')
    if GPUS:
        try:
            for GPU in GPUS:
                tf.config.experimental.set_memory_growth(GPU, True)
                logical_gpus = tf.config.list_logical_devices('GPU')
                print(len(GPUS), "Physical GPUs,", len(logical_gpus), "Logical GPUs") 
        except RuntimeError as  RE:
            print(RE)
    if mp:
        tf.keras.mixed_precision.set_global_policy('mixed_float16')
        print('Mixed precision enabled')
        
tf.random.set_seed(100)
config_gpu(mp=False)

In [ ]:
BATCH_SIZE  = 12
IMG_SIZE    = 256
CHANNELS    = 3
EPOCHS = 10
LR = 0.002

In [ ]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers

# Multi-Atrous Branch
class MultiAtrous(keras.Model):
    def __init__(self, 
                 dilation_rates=[6, 12, 18], 
                 upsampling=1, 
                 kernel_size=3, 
                 padding="same",  **kwargs):
        super(MultiAtrous, self).__init__(name='MultiAtrous', **kwargs)
        self.dilation_rates = dilation_rates
        self.kernel_size = kernel_size 
        self.upsampling = upsampling
        self.padding = padding
      
        # Dilated Convolutions                     
        self.dilated_convs = [
                                layers.Conv2D(
                                    filters       = int(1024 / 4), 
                                    kernel_size   = self.kernel_size,  
                                    padding       = self.padding, 
                                    dilation_rate = rate
                                ) for rate in self.dilation_rates
                             ]
        
        # Global Average Pooling Branch 
        self.gap_branch = keras.Sequential(
            [
                layers.GlobalAveragePooling2D(keepdims=True),
                layers.Conv2D(int(1024 / 2), kernel_size=1),
                layers.Activation('relu'),
                layers.UpSampling2D(size=self.upsampling, interpolation="bilinear")
            ] , name='gap_branch'
        )
        
    def call(self, inputs, training=None, **kwargs):
        local_feature = []

        for dilated_conv in self.dilated_convs:
            x = dilated_conv(inputs) 
            x = self.gap_branch(x)
            local_feature.append(x)
            
        return tf.concat(local_feature, axis=-1)
    def get_config(self):
        config = {
            'dilation_rates': self.dilation_rates,
            'kernel_size'   : self.kernel_size,
            'padding'       : self.padding,
            'upsampling'    : self.upsampling
        }
        base_config = super(MultiAtrous, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
# DOLG: Local-Branch
class DOLGLocalBranch(keras.Model):
    def __init__(self, IMG_SIZE, **kwargs):
        super(DOLGLocalBranch, self).__init__(name='LocalBranch', **kwargs)
        self.multi_atrous = MultiAtrous(padding='same', upsampling=int(IMG_SIZE/32))
        self.conv1 = layers.Conv2D(1024, kernel_size=1)
        self.conv2 = layers.Conv2D(1024, kernel_size=1, use_bias=False)
        self.conv3 = layers.Conv2D(1024, kernel_size=1)
        self.bn = layers.BatchNormalization()

    def call(self, inputs, training=None, **kwargs):
        # Local Branach + Normalization / Conv-Bn Module 
        local_feat = self.multi_atrous(inputs)
        local_feat = self.conv1(local_feat)
        local_feat = tf.nn.relu(local_feat)
        
        # Self-Attention
        local_feat = self.conv2(local_feat)
        local_feat = self.bn(local_feat)

        # l-2 norms
        norm_local_feat = tf.math.l2_normalize(local_feat)

        # softplus activations
        attn_map = tf.nn.relu(local_feat)
        attn_map = self.conv3(attn_map)
        attn_map = keras.activations.softplus(attn_map) 

        # Output of the Local-Branch 
        return  norm_local_feat * attn_map 

In [ ]:
class OrthogonalFusion(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(name='OrthogonalFusion', **kwargs)
    def call(self, inputs):
        local_feat, global_feat = inputs
        height = local_feat.shape[1]
        width  = local_feat.shape[2]
        depth  = local_feat.shape[3]
    
        local_feat = tf.reshape(local_feat, [-1, height*width, depth])
        local_feat = tf.transpose(local_feat, perm=[0, 2, 1])
        
        projection = tf.matmul(
            tf.expand_dims(global_feat, axis=1), 
            local_feat
        )
        projection = tf.matmul(
            tf.expand_dims(global_feat, axis=2),
            projection
        )
        projection = tf.reshape(projection, [-1, height, width, depth])
        
        global_feat_norm = tf.norm(global_feat, ord=2, axis=1)  
        projection = projection / tf.reshape(global_feat_norm*global_feat_norm, shape=[-1, 1, 1, 1])
        local_feat = tf.transpose(local_feat, perm=[0, 1, 2])
        local_feat = tf.reshape(local_feat, [-1, height, width, depth])
    
        orthogonal_comp = local_feat - projection
        global_feat = tf.expand_dims(tf.expand_dims(global_feat, axis=1), axis=1)
        global_feat = tf.broadcast_to(global_feat, tf.shape(local_feat))
        output = tf.concat([global_feat, orthogonal_comp], axis=-1)
        return output


In [ ]:
class GeneralizedMeanPooling2D(layers.Layer):
    def __init__(self, init_norm=3.0, normalize=False, epsilon=1e-6, **kwargs):
        self.init_norm = init_norm
        self.normalize = normalize
        self.epsilon   = epsilon
        super(GeneralizedMeanPooling2D, self).__init__(**kwargs)

    def build(self, input_shape):
        self.p = self.add_weight(name="norms", 
                                 shape=(input_shape[-1],),
                                 initializer=keras.initializers.constant(self.init_norm),
                                 trainable=True)
        super(GeneralizedMeanPooling2D, self).build(input_shape)

    def call(self, inputs):
        x = tf.abs(tf.maximum(self.epsilon, inputs))
        x = tf.pow(x, self.p)
        x = tf.reduce_mean(x, axis=[1,2], keepdims=False) 
        x = tf.pow(x, (1.0 / self.p))
        if self.normalize:
            x = tf.nn.l2_normalize(x, 1)
        return x

    def get_config(self):
        config = {
            'init_norm' : self.init_norm,
            'normalize' : self.normalize,
            'epsilon'   : self.epsilon
        }
        base_config = super(GeneralizedMeanPooling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
base = keras.applications.ResNet50(
            include_top=False,
            weights='imagenet',
            input_tensor=Input((IMG_SIZE, IMG_SIZE, 3))
        )

new_base = keras.Model(
    [base.inputs], 
    [
        base.get_layer('conv3_block4_out').output,  # fol local branch 
        base.get_layer('conv5_block3_out').output   # for global branch 
        ], 
    name='ResNet'
)

In [ ]:
class DOLGNet(keras.Model):
    def __init__(self, backbone=None, num_classes=1, activation=None, **kwargs):
        super(DOLGNet, self).__init__(name='DOLGNet', **kwargs)
        # Number of classes 
        self.num_classes = num_classes
        self.activation  = activation
        
        # Base blcoks 
        self.base = backbone
        self.base_input_shape  = self.base.input_shape[0][1]

        # Top building blocks 
        self.orthogonal_fusion = OrthogonalFusion()
        self.local_branch      = DOLGLocalBranch(IMG_SIZE=self.base_input_shape)
        
        # Tail blcok 1 
        self.glob_branch_pool = keras.Sequential(
            [
                GeneralizedMeanPooling2D(),
                layers.Dense(1024, activation=None)
            ], 
            name='GlobalBranchPooling'
        )
        
        # Head block
        self.classifier = keras.Sequential(
            [
                layers.GlobalAveragePooling2D(name='HeadGAP'),
                layers.Dense(self.num_classes, activation = self.activation)
            ], 
            name='Classifiers'
        )
       
    # forwarding the computation 
    def call(self, inputs, training=None, **kwargs):
        # Get tensor from target layers 
        to_local, to_global = self.base(inputs)

        # Pass the received tensor to Top building blocks 
        local_feat      = self.local_branch(to_local)
        global_feat     = self.glob_branch_pool(to_global)
        orthogonal_feat = self.orthogonal_fusion([local_feat, global_feat]) 
        return self.classifier(orthogonal_feat)

    def build_graph(self):
        x = keras.Input(shape=(self.base_input_shape, self.base_input_shape, 3))
        return keras.Model(inputs=[x], outputs=self.call(x))


In [ ]:
NUM_CLASSES=100
dolg_net = DOLGNet(new_base, num_classes=NUM_CLASSES, activation='softmax')
dolg_net.build_graph().summary()

In [ ]:
display(tf.keras.utils.plot_model(dolg_net.build_graph(), 
                                  show_shapes=True, show_layer_names=True, expand_nested=False))

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os

input_dir = os.path.join('..', 'input')
data_dir = os.path.join(input_dir, 'landmark-recognition-2021')
train_label_info = os.path.join(data_dir, 'train.csv')
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
traincsv = pd.read_csv(train_label_info)
traincsv['path'] = traincsv['images'].transform(lambda x: train_dir+'/'+str(x[0])+'/'+str(x[1])+'/'+
                                            str(x[2])+'/'+str(x)+'.jpg')
topten = traincsv['landmark_id'].value_counts().head(100).index
dftrain = traincsv.loc[traincsv['landmark_id'].isin(topten)].copy()
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dftrain['label'] = encoder.fit_transform(dftrain['landmark_id'])
from sklearn.model_selection import train_test_split
train, val = train_test_split(dftrain, test_size=0.2)
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True)
    

train_data = datagen.flow_from_dataframe(train, x_col='path', y_col='label', class_mode='raw',
                                             target_size=(256,256))

val_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.).flow_from_dataframe(
                    val, x_col='path', y_col='label', class_mode='raw', target_size=(256,256))

In [ ]:
from tensorflow.keras import optimizers, metrics, losses

dolg_net.compile(
    optimizer='Adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'])
dolg_net.fit(train_data, epochs=10, validation_data=val_data, verbose=1)